In [1]:
for i in range(1, 20):
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [1]:
from typing import List, Optional
import json

from pydantic import BaseModel
from groq import Groq

groq = Groq()


# Data model for LLM to generate
class Ingredient(BaseModel):
    name: str
    quantity: str
    quantity_unit: Optional[str]


class Recipe(BaseModel):
    recipe_name: str
    ingredients: List[Ingredient]
    directions: List[str]


def get_recipe(recipe_name: str) -> Recipe:
    chat_completion = groq.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "You are a recipe database that outputs recipes in JSON.\n"
                # Pass the json schema to the model. Pretty printing improves results.
                f" The JSON object must use the schema: {json.dumps(Recipe.model_json_schema(), indent=2)}",
            },
            {
                "role": "user",
                "content": f"Fetch a recipe for {recipe_name}",
            },
        ],
        model="llama3-8b-8192",
        temperature=0,
        # Streaming is not supported in JSON mode
        stream=False,
        # Enable JSON mode by setting the response format
        response_format={"type": "json_object"},
    )
    return Recipe.model_validate_json(chat_completion.choices[0].message.content)


def print_recipe(recipe: Recipe):
    print("Recipe:", recipe.recipe_name)

    print("\nIngredients:")
    for ingredient in recipe.ingredients:
        print(
            f"- {ingredient.name}: {ingredient.quantity} {ingredient.quantity_unit or ''}"
        )
    print("\nDirections:")
    for step, direction in enumerate(recipe.directions, start=1):
        print(f"{step}. {direction}")


recipe = get_recipe("apple pie")
print_recipe(recipe)


Recipe: Apple Pie

Ingredients:
- All-purpose flour: 2 1/4 cups cups
- Cold unsalted butter: 1 cup cups
- Granulated sugar: 1/2 cup cups
- Salt: 1/4 teaspoon teaspoons
- Ground cinnamon: 1/2 teaspoon teaspoons
- Ground nutmeg: 1/4 teaspoon teaspoons
- Large eggs: 2 count
- Apple cider vinegar: 1 tablespoon tablespoons
- Apples, peeled and sliced: 6-8 count

Directions:
1. Preheat oven to 375°F (190°C).
2. Make the crust: In a large bowl, whisk together flour, salt, and sugar. Add butter and use a pastry blender or your fingers to work it into the flour until the mixture resembles coarse crumbs.
3. Make the filling: In a separate bowl, whisk together eggs, apple cider vinegar, cinnamon, and nutmeg. Add the sliced apples and toss until they're evenly coated.
4. Assemble the pie: Roll out the crust on a lightly floured surface to a thickness of about 1/8 inch. Transfer the dough to a 9-inch pie dish and trim the edges to fit.
5. Fill the pie: Pour the apple mixture into the pie crust and 

In [8]:
json.dumps()

TypeError: dumps() missing 1 required positional argument: 'obj'